### Packages

In [1]:
using DynamicalSystems, DifferentialEquations, OrdinaryDiffEq, StaticArrays
using CairoMakie

### Functions

In [2]:
function model(u, p ,t)
    
    E, x, u_, y = u
    τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0 = p
    
    U(y, U0, ΔU0, ythr) = U0 + ΔU0 / ( 1 + exp( -50 * ( y - ythr ) ) )
    σ(x, xthr)= 1 / (1 + exp(-20 * (x - xthr)));
    
    du1 = (-E + α * log( 1 + exp( ( J * u_ * x * E + I0 ) / α ) )) / τ
    du2 = ( 1 - x ) / τ_D - u_ * x * E
    du3 = (U(y, U0, ΔU0, ythr) - u_) / τ_F + U(y, U0, ΔU0, ythr) * ( 1 - u_ ) * E
    du4 = -y /  τ_y + β * σ(x, xthr)
    
    return SVector(du1, du2, du3, du4)
    
end;

function output(index_i, index_j, U0_lc, I0_lc, u0_lc)
        println("index i: $index_i"); flush(stdout)
        println("index j: $index_j"); flush(stdout)
        println("U0: $U0_lc;"); flush(stdout)
        println("I0: $I0_lc;"); flush(stdout)
        println("u0: $u0_lc;"); flush(stdout)
end
function separate()
    println(" --------------------- "); flush(stdout)
    println("  "); flush(stdout)
end

function parabola(x)
    a = 0.13625841750841752
    b = 0.189709595959596
    c = 0.22
    return  a * (x.^2) + b * x .+ c 
end

function line(x)
    a = -0.2558139534883721
    b = -0.16046511627906984
    return a * x .+ b
end

function print_output(index, I0, U0, initial_condition)
    println("index: $index");flush(stdout)
    println("I0: $I0");flush(stdout)
    println("U0: $U0");flush(stdout)
    println("initial condition: $initial_condition")
end
function separate()
    println("_________________")
    println("")
end

separate (generic function with 1 method)

### Parametres

In [3]:
τ = 0.013; τ_D = 0.15; τ_y = 1.8; τ_F = 1;
α = 1.5; J = 3.07; β = 0.4375
ΔU0 = 0.305
xthr = 0.9; ythr = 0.5
parametres = SA[ τ, α, τ_D, J, NaN, ΔU0, τ_y, β, xthr, ythr, NaN ]
timefinal = 3000.0
println(parametres)

[0.013, 1.5, 0.15, 3.07, NaN, 0.305, 1.8, 0.4375, 0.9, 0.5, NaN]


### Grid

In [4]:
startI = -1.8
endI = -1.32
lengthI = 100
rangeI = range( startI, endI, length = 100 )
step_to_left = rangeI.step.hi

0.004848484848484658

In [5]:
file = "2d_map_betweeen_line_parabola.txt"

"2d_map_betweeen_line_parabola.txt"

In [ ]:
for (index_I0, I0) in enumerate(rangeI)
    
        if index_I0 == 1
            global initial_condition_function = SA[0.0, 0.2, 0.1, 0.1]
        end
        
        U0 = parabola(I0)
    
        print_output(index_I0, I0, U0, initial_condition_function)
    
        parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0  ]
    
        ds = ContinuousDynamicalSystem( model, initial_condition_function, parametres )
    
        tr = trajectory(ds, timefinal / 2, Ttr = timefinal / 4, Δt = 0.001;
                    diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
        initial_condition_function = tr[end, :]
    
        println("last point: $initial_condition_function");flush(stdout)
    
        λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal / 4, Δt = 1;
                            diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
        println("LSE: $λλ");flush(stdout)

        open(file, "a") do io
            println(io, " $I0 $U0 $(λλ[1])")
        end
    
        println("----------------------")
        if I0  >  -1.8
            move_to_left( I0, U0, initial_condition_function, step_to_left )
        end
        
end

index: 1
I0: -1.8
U0: 0.32000000000000006
initial condition: [0.0, 0.2, 0.1, 0.1]
last point: [1.7303735312129922, 0.8546534767458698, 0.6819520373729314, 0.09103793623701394]
LSE: [0.00045279931611982827, -0.4584742933804628, -0.5556136735610765, -8.704974692499281]
----------------------
index: 2
I0: -1.7951515151515152
U0: 0.31854467849601
initial condition: [1.7303735312129922, 0.8546534767458698, 0.6819520373729314, 0.09103793623701394]
last point: [1.0486697820068387, 0.6124287315138137, 0.7774131582109282, 0.08304585378161278]
LSE: [0.00036172332160014446, -0.5167413091914771, -0.5553448575277006, -8.692111743781584]
----------------------

<<<<<<<<<<<<<<<<<<<<<
 <- 
index: 1
I0: -1.7999999999999998
U0: 0.31854467849601
initial condition: [1.0486697820068387, 0.6124287315138137, 0.7774131582109282, 0.08304585378161278]
last point: [0.9150143815319376, 0.9247912913711375, 0.5925230111024512, 0.4894138343536877]
LSE: [-0.3347300117843138, -0.33452609000071026, -12.242345234710735,

In [7]:
function move_to_left( I0, U0, initial_condition_function, step )
    
    println("");flush(stdout)
    println("<<<<<<<<<<<<<<<<<<<<<");flush(stdout)
    index_left = 1
    
    while line(I0) < U0 && I0 > -1.8
        
        if index_left == 1
            global initial_condition_left = initial_condition_function
            I0 = I0 - step
        end
        
        println(" <- ")
        print_output(index_left, I0, U0, initial_condition_left)
        
        parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0  ]
        
        ds = ContinuousDynamicalSystem( model, initial_condition_left, parametres )
    
        tr = trajectory(ds, timefinal / 2, Ttr = timefinal / 4, Δt = 0.001;
                            diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
        initial_condition_left = tr[end, :]
        
        println("last point: $initial_condition_left");flush(stdout)
        λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal / 2, Δt = 1;
                                diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
        println("LSE: $λλ");flush(stdout)

        open(file, "a") do io
            println(io, " $I0 $U0 $(λλ[1])")
        end
        separate()
        I0 = I0 - step
        index_left += 1
    end
end

move_to_left (generic function with 1 method)